In [2]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Polygon, Point

In [14]:
df = pd.read_csv('poligonos_mx.csv') #Se lee el csv como un dataframe normal y después se convierte a geodataframe
df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs='epsg:4326')
gdf.head()

,Unnamed: 0,estado,geometry
0,0,ZAC,"POLYGON ((-100.83230 24.55490, -100.85400 24.5..."
1,1,YUC,"POLYGON ((-87.53860 21.48640, -87.52560 21.287..."
2,2,VER,"POLYGON ((-97.79320 22.24960, -97.76810 22.049..."
3,3,TLA,"POLYGON ((-98.58490 19.46750, -98.63610 19.481..."
4,4,TAM,"POLYGON ((-97.79320 22.24960, -97.87290 22.197..."


In [4]:
df_miso = pd.read_csv('miso_Septiembre2017_completo_geo.csv')
df_miso.head()

,index,id,user,datetime,text,lat,lon,klass,language,misoginia
0,1,9.138147e+17,julietalitzel,2017-09-29T17:16:23.000Z,Hay que salirnos de la clase de francés @1617...,19.032229,-98.154267,positivo,Español,{'loca': 1}
1,2,9.139210e+17,klaukastro,2017-09-30T00:18:44.000Z,Quiero a la vieja Siri de regreso 😫😫😫 https://...,19.318697,-99.305581,negativo,Español,{'vieja': 1}
2,3,9.141734e+17,sannthi,2017-09-30T17:01:38.000Z,Emprendiendo una nueva aventura ... #playaDelC...,20.750010,-100.099640,positivo,Español,{'aventura': 1}
3,4,9.140723e+17,trendinaliaMX,2017-09-30T10:19:48.000Z,⒍ #SabadoDeGanarSeguidores ⒎ Ferrari ⒏ Lewis H...,19.431900,-99.132900,neutro,Español,{'India': 1}
4,5,9.141989e+17,OLENKA666,2017-09-30T18:42:56.000Z,"Detrás de una gran mujer, hay una amiga loca q...",19.691509,-101.150063,positivo,Español,{'loca': 1}


In [15]:
len(df_miso)

470

In [16]:
columnas = ['id','user','datetime','text','lat','lon','klass','language','misoginia', 'estado']
df_miso_estado = pd.DataFrame(columns = columnas)
df_miso_estado.head()

,id,user,datetime,text,lat,lon,klass,language,misoginia,estado


In [7]:
def dime_estado(lon, lat):
    estado = 'Fuera de México'
    band = 0
    point = Point(lon, lat)
    for index, row in gdf.iterrows():
        poligon = gdf['geometry'][index]
        if poligon.contains(point):
            estado = gdf['estado'][index]
            band = 1
            break
    return estado

In [20]:
lat = 17.5482618
lon = -95.2724035
estado = dime_estado(lon, lat)
estado

'OAX'

In [10]:
estados=[]
for index, row in df_miso.iterrows():
    ide = row['id']
    user = row['user']
    datetime = row['datetime']
    text = row['text']
    lat = row['lat']
    lon = row['lon']
    klass = row['klass']
    language = row['language']
    misoginia = row['misoginia']
    estado = dime_estado(lon, lat)
    estados.append(estado)


In [11]:
df_miso_estado['estado'] = estados
df_miso_estado.head()

,id,user,datetime,text,lat,lon,klass,language,misoginia,estado
0,9.138147e+17,julietalitzel,2017-09-29T17:16:23.000Z,Hay que salirnos de la clase de francés @1617...,19.032229,-98.154267,positivo,Español,{'loca': 1},PUE
1,9.139210e+17,klaukastro,2017-09-30T00:18:44.000Z,Quiero a la vieja Siri de regreso 😫😫😫 https://...,19.318697,-99.305581,negativo,Español,{'vieja': 1},MX
2,9.141734e+17,sannthi,2017-09-30T17:01:38.000Z,Emprendiendo una nueva aventura ... #playaDelC...,20.750010,-100.099640,positivo,Español,{'aventura': 1},QUE
3,9.140723e+17,trendinaliaMX,2017-09-30T10:19:48.000Z,⒍ #SabadoDeGanarSeguidores ⒎ Ferrari ⒏ Lewis H...,19.431900,-99.132900,neutro,Español,{'India': 1},CMX
4,9.141989e+17,OLENKA666,2017-09-30T18:42:56.000Z,"Detrás de una gran mujer, hay una amiga loca q...",19.691509,-101.150063,positivo,Español,{'loca': 1},MIC


In [12]:
df_miso_estado.tail()

,id,user,datetime,text,lat,lon,klass,language,misoginia,estado
465,9.139272e+17,meryvasquez12,2017-09-30T00:43:27.000Z,ay me hacen sentir de lo más culera y no lo so...,15.772467,-90.224221,negativo,Español,{'culera': 1},Fuera de México
466,9.139312e+17,lumonts,2017-09-30T00:59:11.000Z,"Este pedo esta muy feo, si ne duermo me levant...",16.742435,-93.139659,negativo,Español,{'jodida': 1},CHP
467,9.141401e+17,cynthiaacosta96,2017-09-30T14:49:12.000Z,Mi amiguita Maryel me trajo taquitos 😍❤️,25.736715,-100.270474,positivo,Español,{'amiguita': 1},NLE
468,9.138247e+17,MyrnaRockera,2017-09-29T17:56:03.000Z,La vieja Tegucigalpa en los tiempos de Froylán...,15.201132,-86.243194,positivo,Español,{'vieja': 1},Fuera de México
469,9.102521e+17,Hraman_yd,2017-09-19T21:19:58.000Z,#Earthquake. Condolence to Mexico City people ...,28.638489,77.095086,neutro,Español,{'India': 1},Fuera de México


In [13]:
df_miso_estado = df_miso_estado[df_miso_estado.estado != 'Fuera de México']
df_miso_estado.tail()

,id,user,datetime,text,lat,lon,klass,language,misoginia,estado
461,9.142098e+17,PincheRodriGoe,2017-09-30T19:26:23.000Z,FAV si les cuento una historia muy cagada.,20.551409,-99.955803,negativo,Español,{'cagada': 1},QUE
462,9.141822e+17,AmelieHarker,2017-09-30T17:36:23.000Z,@yolandandrade @Mojoetv @Montserrat33 Claro! N...,19.183953,-96.229396,positivo,Español,{'loca': 1},VER
464,9.138399e+17,CSaizar,2017-09-29T18:56:15.000Z,"Muchas felicidades, querida @Margo_Glantz . Un...",19.318042,-99.248009,positivo,Español,{'querida': 1},CMX
466,9.139312e+17,lumonts,2017-09-30T00:59:11.000Z,"Este pedo esta muy feo, si ne duermo me levant...",16.742435,-93.139659,negativo,Español,{'jodida': 1},CHP
467,9.141401e+17,cynthiaacosta96,2017-09-30T14:49:12.000Z,Mi amiguita Maryel me trajo taquitos 😍❤️,25.736715,-100.270474,positivo,Español,{'amiguita': 1},NLE
